# 🥭 Context

A `Context` object to manage Solana connection configuration and Mango groups.

In [ ]:
import os
import typing

from decimal import Decimal
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.rpc.types import MemcmpOpts, TokenAccountOpts
from solana.rpc.commitment import Single

from Constants import MangoConstants, SOL_DECIMAL_DIVISOR


In [ ]:
class Context:
    def __init__(self, cluster: str, cluster_url: str, program_id: PublicKey, dex_program_id: PublicKey,
                 group_name: str, group_id: PublicKey):
        self.cluster: str = cluster
        self.cluster_url: str = cluster_url
        self.client: Client = Client(cluster_url)
        self.program_id: PublicKey = program_id
        self.dex_program_id: PublicKey = dex_program_id
        self.group_name: str = group_name
        self.group_id: PublicKey = group_id
        self.commitment = Single
        self.encoding:str = "base64"


    def fetch_sol_balance(self, account_public_key: PublicKey):
        result = self.client.get_balance(account_public_key, commitment=self.commitment)
        value = Decimal(result["result"]["value"])
        return value / SOL_DECIMAL_DIVISOR


    def fetch_token_balance(self, account_public_key: PublicKey, token_public_key: PublicKey):
        opts = TokenAccountOpts(mint = token_public_key)

        token_account = self.client.get_token_accounts_by_owner(account_public_key, opts, commitment=self.commitment)
        result = self.client.get_token_account_balance(token_account["result"]["value"][0]["pubkey"], commitment=self.commitment)
        value = Decimal(result["result"]["value"]["amount"])
        decimal_places = result["result"]["value"]["decimals"]
        divisor = Decimal(10 ** decimal_places)
        return value / divisor


    # This should be easier to call and should probably be on the Client object, but we don't
    # control that.
    def fetch_program_accounts_for_owner(self, program_id: PublicKey, owner: PublicKey):
        memcmp_opts = [
            MemcmpOpts(offset=40, bytes=str(owner)),
        ]

        return self.client.get_program_accounts(program_id, memcmp_opts=memcmp_opts, commitment=self.commitment, encoding=self.encoding)


    @staticmethod
    def _lookup_name_by_address(address: PublicKey, collection: typing.Dict[str, str]) -> typing.Optional[str]:
        address_string = str(address)
        for stored_name, stored_address in collection.items():
            if address_string == stored_address:
                return stored_name
        return None

    def lookup_market_name(self, market_address: PublicKey) -> str:
        return Context._lookup_name_by_address(market_address, MangoConstants[self.cluster]["spot_markets"]) or "« Unknown Market »"

    def lookup_oracle_name(self, token_address: PublicKey) -> str:
        return Context._lookup_name_by_address(token_address, MangoConstants[self.cluster]["oracles"]) or "« Unknown Oracle »"

    def lookup_token_name(self, token_address: PublicKey) -> str:
        return Context._lookup_name_by_address(token_address, MangoConstants[self.cluster]["symbols"]) or "« Unknown Token »"

    def __str__(self) -> str:
        return f"""« Context:
    Cluster: {self.cluster}
    Cluster URL: {self.cluster_url}
    Program ID: {self.program_id}
    DEX Program ID: {self.dex_program_id}
    Group Name: {self.group_name}
    Group ID: {self.group_id}
"""

    def __repr__(self) -> str:
        return f"{self}"


# default_context object

A default `Context` object that connects to mainnet, to save having to create one all over the place.

In [ ]:
default_cluster = os.environ.get("CLUSTER") or 'mainnet-beta'
default_cluster_url = os.environ.get("CLUSTER_URL") or MangoConstants["cluster_urls"][default_cluster]

default_program_id = PublicKey(MangoConstants[default_cluster]["mango_program_id"])
default_dex_program_id = PublicKey(MangoConstants[default_cluster]["dex_program_id"])

default_group_name = os.environ.get("GROUP_NAME") or 'BTC_ETH_USDT'
default_group_id = PublicKey(MangoConstants[default_cluster]["mango_groups"][default_group_name]["mango_group_pk"])

default_context = Context(default_cluster, default_cluster_url, default_program_id,
                          default_dex_program_id, default_group_name, default_group_id)

# Running

If running interactively, just print out the default Context object.

In [ ]:
if __name__ == "__main__":
    print(default_context)

    print("Lookup ETH token name result:", default_context.lookup_token_name(PublicKey("2FPyTwcZLUg1MDrwsyoP4D6s1tM7hAkHYRjkNb5w6Pxk")))
    print("Lookup BTC/USDC market name result:", default_context.lookup_market_name(PublicKey("CVfYa8RGXnuDBeGmniCcdkBwoLqVxh92xB1JqgRQx3F")))
    # Fill out your account address between the quotes below
    MY_ACCOUNT_ADDRESS = ""
    # Don't edit anything beyond here.

    if MY_ACCOUNT_ADDRESS != "":
        account_key = PublicKey(MY_ACCOUNT_ADDRESS)
        print("SOL balance:", default_context.fetch_sol_balance(account_key))
        print("SRM balance:", default_context.fetch_token_balance(account_key, PublicKey("SRMuApVNdxXokk5GT7XD5cUUgXMBCoAz2LHeuAoKWRt")))
